In [1]:
import sys
sys.path.append("..")  # <-  This should point to the root directory of the project relative to this file

from custom_scripts import database
from custom_scripts import preprocessing
from custom_scripts import preparation
from custom_scripts import modeling
import pandas as pd
import numpy as np

In [27]:
raw_test_flights = preprocessing.get_test_flights()
average_flight_delays = pd.read_csv('../data/preprocessing/average_flight_delays_by_flight_number.csv')
# average_destination_delays = pd.read_csv('../data/preprocessing/avg_flight_delays_by_destination_airport.csv')
average_origin_delays = pd.read_csv('../data/preprocessing/avg_flight_delays_by_origin_airport.csv')
average_carrier_delays = pd.read_csv('../data/preprocessing/avg_flight_delays_by_carrier.csv')
average_day_of_year_delays = pd.read_csv('../data/preprocessing/avg_flight_delays_by_day_of_year.csv')


In [42]:
test_flights = raw_test_flights.copy()
print(test_flights.shape)
test_flights.head(1)

(150623, 12)


,fl_date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance
0,2020-01-01,WN,N951WN,5888,13891,14771,1810,1945,95,N,1,363


In [43]:
test_flights['fl_date'] = test_flights['fl_date'].dt.dayofyear
test_flights = test_flights.rename(columns={'fl_date':'day_of_year'})
print(test_flights.shape)
test_flights.head(1)


(150623, 12)


,day_of_year,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance
0,1,WN,N951WN,5888,13891,14771,1810,1945,95,N,1,363


In [44]:
### Historic average delay features
fl_num_delays = average_flight_delays[['op_carrier_fl_num','avg_arr_delay']]
test_flights = pd.merge(test_flights, fl_num_delays, on='op_carrier_fl_num')
test_flights.rename(columns={'avg_arr_delay':'avg_fl_num_arr_delay'}, inplace=True)

dest_dep_delays = average_origin_delays[['origin_airport_id', 'avg_dep_delay']]
test_flights = pd.merge(test_flights, dest_dep_delays, on='origin_airport_id')
test_flights.rename(columns={'avg_dep_delay':'avg_origin_dep_delay'}, inplace=True)

carrier_delays = average_carrier_delays[['op_unique_carrier', 'avg_carrier_delay']]
test_flights = pd.merge(test_flights, carrier_delays, on='op_unique_carrier')
test_flights.rename(columns={'avg_carrier_delay':'carrier_avg_carrier_delay'}, inplace=True)

day_of_year_delays = average_day_of_year_delays[['day_of_year','avg_arr_delay', 'avg_weather_delay']]
test_flights = pd.merge(test_flights,day_of_year_delays, on='day_of_year' )
test_flights.rename(columns={'avg_arr_delay':'day_avg_delay', 
                             'avg_weather_delay':'day_avg_weather_delay'}, inplace=True)

test_flights = test_flights.rename(columns={'avg_arr_delay':'avg_fl_num_arr_delay',
                                            'avg_dep_delay': 'avg_origin_dep_delay', })
print(test_flights.shape)
test_flights.head(1)

(150623, 17)


,day_of_year,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,dup,flights,distance,avg_fl_num_arr_delay,avg_origin_dep_delay,carrier_avg_carrier_delay,day_avg_delay,day_avg_weather_delay
0,1,WN,N951WN,5888,13891,14771,1810,1945,95,N,1,363,8.75618,7.605131,16.74029,10.488622,3.270192


In [40]:
average_day_of_year_delays

,day_of_year,avg_arr_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay,avg_dep_delay
0,1.0,10.488622,24.226509,3.270192,9.308377,28.867853,16.427019
1,2.0,9.968470,20.181292,3.097133,9.627948,27.497326,15.290052
2,3.0,4.861474,21.895186,2.675742,8.769455,24.413181,11.164983
3,4.0,-0.225516,23.994110,5.239463,8.960795,22.033867,7.009716
4,5.0,7.331032,25.348407,4.181409,17.958879,29.420204,13.643319
...,...,...,...,...,...,...,...
360,361.0,10.692640,20.433123,4.743057,10.104277,30.129068,16.406279
361,362.0,16.816636,20.640679,3.894288,13.002324,31.244876,21.610000
362,363.0,9.349526,22.756895,3.178721,8.778265,28.472237,15.625149
363,364.0,7.604898,20.687131,3.865298,12.472308,28.440461,13.428252
